In [25]:
import pandas as pd

df = pd.read_json('data/human_annotations/cnn_human.json', orient='index')

In [10]:
df["gpt3"].values[0]['text']

"The article discusses Russian president Vladimir Putin's speech on Victory Day, in which he blended history with the present and justified his war. It is speculated that Putin may use this day to escalate the conflict in Ukraine, but it is also noted that he has few successes in Ukraine to brag of. The question now is whether Putin will use this day to escalate the war in other ways."

In [30]:
df = pd.read_json('data/test-500/dm.json', orient='index')

In [24]:
df[['id', 'article', "reference", "gpt3"]].to_csv('data/validation_set/xsum.csv', index=False)

In [29]:
df[['id', 'article', "reference", "gpt3"]].to_csv('data/validation_set/cnn.csv', index=False)

In [32]:
df[["id", "article", "reference", "gpt3"]].to_csv('data/validation_set/dm.csv', index=False)

In [34]:
df = pd.read_json('data/validation_set/writer_summaries.json')

In [44]:
df.to_csv('data/validation_set/cnndm_xsum_writer_summaries.csv', index=False)

In [46]:
df_grouped = df.groupby(['article_id', 'article']).apply(lambda x: x.loc[x.summary.str.len().idxmax()]).reset_index(drop=True)

In [47]:
import json

# Define function to create a JSON object for each row
def create_json(row):
    return {
        "vicuna_format": {
            "conversations": [
                {
                    "from": "human",
                    "value": f"Summarize the text below delimited by triple backquotes (```): \n```{row['article']}```\nSUMMARY:"
                },
                {
                    "from": "gpt",
                    "value": row['summary']
                }
            ]
        }
    }

# Apply function to each row in the DataFrame
json_list = df_grouped.apply(create_json, axis=1).tolist()

In [48]:
with open('data/validation_set/cnndm_xsum_writer_summaries.jsonl', 'w') as f:
    for item in json_list:
        f.write(json.dumps(item) + '\n')

In [52]:
df_xsum = pd.read_csv('data/validation_set/xsum.csv')
df_cnn = pd.read_csv('data/validation_set/cnn.csv')
df_dm = pd.read_csv('data/validation_set/dm.csv')
df_samsum = pd.read_csv('data/validation_set/samsum.csv')

In [53]:
df = pd.concat([df_cnn, df_dm, df_xsum])

In [59]:
df.head()

,id,article,reference,gpt3
0,dae2675302d92bdf0bbd6d35c3e473389f8bb5a1,(CNN) Two CNN Heroes are among the earthquake ...,Anuradha Koirala and 425 young women and girls...,Two CNN heroes who are working in Nepal after ...
1,a8abd023237f7a7f4f83a08694de3d067a24fe95,Seoul (CNN) South Korea's Prime Minister Lee W...,Calls for Lee Wan-koo to resign began after So...,Prime Minister Lee Wan-koo offered to resign o...
2,cd608e32768d6dcf6fccdc15e4052619f2bfc58c,"Sao Paulo, Brazil (CNN) Brazilian police have ...",A top official with President Dilma Rousseff's...,Brazilian police have arrested the treasurer o...
3,fc1813a2c4ac4a320c1874e5974a09cea9c9a381,(CNN) Most kids want to go out and play when t...,Zuriel Oduwole is a 12-year-old filmmaker . To...,Zuriel Oduwole is a 12-year-old filmmaker from...
4,43f18fc9b20fe64405d7f18a08fb8d9ef932ed8b,(CNN) A mammoth fire broke out Friday morning ...,Fire breaks out at the General Electric Applia...,A fire broke out at the General Electric Appli...


In [60]:
def create_json(row):
    return {
        "vicuna_format": {
            "conversations": [
                {
                    "from": "human",
                    "value": f"Summarize the text below delimited by triple backquotes (```): \n```{row['article']}```\nSUMMARY:"
                },
                {
                    "from": "gpt",
                    "value": row['gpt3']
                }
            ]
        }
    }

# Apply function to each row in the DataFrame
json_list = df.apply(create_json, axis=1).tolist()

In [61]:
with open('data/validation_set/cnndm_xsum_gpt3.jsonl', 'w') as f:
    for item in json_list:
        f.write(json.dumps(item) + '\n')

In [62]:
df_samsum.head()

,id,dialogue,summary
0,13817023,"A: Hi Tom, are you busy tomorrow’s afternoon?\...",A will go to the animal shelter tomorrow to ge...
1,13716628,Emma: I’ve just fallen in love with this adven...,Emma and Rob love the advent calendar. Lauren ...
2,13829420,Jackie: Madison is pregnant\nJackie: but she d...,Madison is pregnant but she doesn't want to ta...
3,13819648,Marla: <file_photo>\nMarla: look what I found ...,Marla found a pair of boxers under her bed.
4,13728448,Robert: Hey give me the address of this music ...,Robert wants Fred to send him the address of t...


In [63]:
def create_json(row):
    return {
        "vicuna_format": {
            "conversations": [
                {
                    "from": "human",
                    "value": f"Summarize the conversation below delimited by triple backquotes (```): \n```{row['dialogue']}```\nSUMMARY:"
                },
                {
                    "from": "gpt",
                    "value": row['summary']
                }
            ]
        }
    }

# Apply function to each row in the DataFrame
json_list = df_samsum.apply(create_json, axis=1).tolist()

In [64]:
with open('data/validation_set/samsum_reference.jsonl', 'w') as f:
    for item in json_list:
        f.write(json.dumps(item) + '\n')